# MoI ML CNN Experiment

- Train op fashion mnist
- Laat experimenten zien: kies 2 - 3 params, kies een range en voer tests uit. Doel is 93% accuracy.

In [1]:
import torch
import torch.nn as nn
import seaborn as sns
import sys
from pathlib import Path
sys.path.insert(0, "../")

In [2]:
from src.data import make_dataset
from src.models import imagemodels
from src.models import train_model
import gin


2022-12-01 11:26:34.594537: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 11:26:39.218988: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-01 11:26:39.223398: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-01 11:26:39.223443: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [5]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [6]:
# Load config for this notebook
gin.parse_config_file("cnn.gin")

ParsedConfigFileIncludesAndImports(filename='cnn.gin', imports=[], includes=[])

In [7]:
# Get MNIST data
train_dataloader, test_dataloader = make_dataset.get_MNIST()

In [8]:

# Set some other parameters
import torch.optim as optim
from src.models import metrics
optimizer = optim.Adam
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()
from src.models import train_model

## Voer een experiment uit.


### Configuratie voor het experiment

Sla de gewenste structuur voor de lagen op in een lijst configuraties

In [17]:
from dataclasses import dataclass
@dataclass
class NNLayerConfig:
    """Class for storing config and results for a layer."""
    loss: float
    num_params: int
    filter1_size: int
    filter1_stride: int
    filter1_padding: int
    filter2_size: int
    filter2_stride: int
    filter2_padding: int
    dim_1: int
    dim_2: int

    # TODO calculate input and output dims
    def calc_dim_1():
        return 0
    def calc_dim_2():
        return 0

configs = []  # List of NNLayerConfig objects

# Configuration for the experiment. For now do nothing with stride and padding.
s1 = [1, 2, 3]
s2 = [2, 1, 1]
dim1 = 64
dim2 = 1
for (f1s, f2s) in zip(s1, s2):
    configs.append(NNLayerConfig(   
                                    loss = 0,
                                    num_params = 0,
                                    filter1_size = f1s, 
                                    filter1_stride = 1,
                                    filter1_padding =  0,
                                    filter2_size = f2s,
                                    filter2_stride = 1,
                                    filter2_padding = 0,
                                    dim_1 = dim1,
                                    dim_2 = dim2
                                    ))



In [ ]:



# Define model
class ExpCNN(nn.Module):
    def __init__(self):
        super().__init__()

        # Convolutions are set separately
       
        # Dense is default
        
        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 10)
        )

    def set_convolution(self, conv : nn.Sequential):
        self.convolutions = conv

    def forward(self, x):
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# Loop through some kernel sizes
for ksize in range(2,4):
    conv = nn.Conv2d(
        in_channels=1, 
        out_channels=32,
        kernel_size=ksize,
        padding=(1,1),
        stride=2)
    
    model = ExpCNN().to(device)
    model.set_convolution(nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=ksize, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            # The original:
            # nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2),
            # nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2),
            # nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2),


        ))

    # Now train
    model = train_model.trainloop(
        epochs=2, # was 10
        model=model,
        optimizer=optimizer,
        learning_rate=1e-3,
        loss_fn=loss_fn,
        metrics=[accuracy],
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        log_dir="../models/test/",
        train_steps=len(train_dataloader),
        eval_steps=len(test_dataloader),
    )


Checkout results using tensorboard: 
- run `tensorboard --logdir models` in the terminal
- tensorboard will launch at `localhost:6006` and vscode will notify you that the port is forwarded
- you can either press the `launch` button in VScode or open your local browser at `localhost:6006`